In [ ]:
import os
import cv2
import keras
import random
import numpy as np
import get_proposal
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import  GlobalAveragePooling2D,Dropout,Dense, Flatten
from keras.models import Model

In [ ]:
#extract Dataset
!pip install py7zr
import py7zr
archive = py7zr.SevenZipFile('Dataset.7z', mode='r')
archive.extractall(path="/content/Dataset")
archive.close()

In [ ]:
EPOCHS = 30
BATCH_SIZE= 50
n_classes = 3

In [ ]:
DATA_PATH = '/content/Dataset/'

classes = ['0','1','2']

x_train = []
y_train = []
for cl in classes:
  directory_name = os.path.join(DATA_PATH, cl)
  for filename in os.listdir(directory_name):
    filename_jpg = filename[:-4] + '.jpg'
    filename_str = os.path.join(directory_name, filename_jpg)
    image = mpimg.imread(os.path.join(filename_str))
    x_t,y_t = get_proposal.get_proposals_for_train(image,int(cl))
    
    x_t = np.squeeze(x_t)
    x_train.append(x_t)
    y_train.append(y_t)


In [ ]:
x_train = np.array(x_train)
y_train = keras.utils.to_categorical(y_train, n_classes)

train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    shuffle=True,
)


In [ ]:
def build_model(n):
  base_model = keras.applications.resnet50.ResNet50(
      include_top=False,
      weights=None
  )
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dropout(0.4)(x)
  x = Dense(84, activation= 'sigmoid')(x)
  x = Dropout(0.4)(x)
  predictions = Dense(n, activation= 'softmax')(x)
  model = Model(inputs = base_model.input, outputs = predictions)
  return model

model = build_model(n_classes)
loss = 'categorical_crossentropy'
optimizer = 'adam'
model.compile(loss= loss, optimizer= optimizer, metrics='accuracy')
model.fit(train_generator, epochs=EPOCHS, batch_size=BATCH_SIZE,verbose=True)

model.save('trainedModel',save_format='h5')

Epoch 1/30
55/55 [==============================] - 29s 445ms/step - loss: 0.9894 - accuracy: 0.6222
Epoch 2/30
55/55 [==============================] - 25s 456ms/step - loss: 0.7306 - accuracy: 0.6975
Epoch 3/30
55/55 [==============================] - 26s 466ms/step - loss: 0.6625 - accuracy: 0.7197
Epoch 4/30
55/55 [==============================] - 27s 487ms/step - loss: 0.6066 - accuracy: 0.7413
Epoch 5/30
55/55 [==============================] - 27s 486ms/step - loss: 0.6078 - accuracy: 0.7314
Epoch 6/30
55/55 [==============================] - 26s 480ms/step - loss: 0.5304 - accuracy: 0.7915
Epoch 7/30
55/55 [==============================] - 27s 484ms/step - loss: 0.5245 - accuracy: 0.7937
Epoch 8/30
55/55 [==============================] - 27s 486ms/step - loss: 0.4990 - accuracy: 0.8027
Epoch 9/30
55/55 [==============================] - 27s 484ms/step - loss: 0.5432 - accuracy: 0.7742
Epoch 10/30
55/55 [==============================] - 27s 482ms/step - loss: 0.5496 - accura

In [ ]:
#making test data

f = open("dataset.txt","w+")

data_path = '/content/Dataset/'
classes = ['0','1','2']
for cl in classes:
  dir_name = os.path.join(data_path, cl)
  filename = random.sample(os.listdir(dir_name),100)
  for i in filename:
    path = os.path.join(dir_name, i)
    f.write(path+"\n")
f.close()

In [ ]:
#evaluate

f = open("dataset.txt", "r")
data = []
for line in f:
  data.append(line.strip())

x_test = []
y_true = []
for i in data:
  image = mpimg.imread(os.path.join(i))
  x_te = get_proposal.get_proposals_for_test(image)
  x_test.append(x_te)
  y_true.append(i[-10])

y_test = y_true
x_test = np.array(x_test)
x_test = np.squeeze(x_test)
y_true = keras.utils.to_categorical(y_true, n_classes)

loaded_model = keras.models.load_model('trainedModel')
pred = loaded_model.predict(x_test)
y_pred = []
p = open("predictions.txt","w+")
for i in range(len(data)):
  y_pred.append(str(np.argmax(pred[i])))
  stri = data[i]+", " + y_pred[i]
  p.write(stri+"\n")

import sklearn
from sklearn import metrics
sklearn.metrics.f1_score(y_test, y_pred, average='macro')

0.7774856506050819